# Features selection with STEPWISE

In [27]:
import sys
my_path = r'C:\Users\utente\Documents\GitHub\Project1_ML'
sys.path.insert(0,my_path + r'\code\COMMON')

import numpy as np 
import matplotlib.pyplot as plt
from proj1_helpers import load_csv_data, predict_labels 
from implementations import *
from outliers import handle_outliers
from labels import idx_2labels
from standard import standardize
from costs import compute_loglikelihood_reg
from optimize_hyperparams import *
from cross_validation import *

## Load data and outliers management

In [28]:
yb, input_data, ids = load_csv_data(my_path + r'/data/train.csv', sub_sample=False)

In [29]:
input_data.shape

(250000, 30)

In [30]:
input_data, Y = handle_outliers(input_data,yb,-999,'mean') # substiution with mean because the standardization
                                                           #can be affected, otherwise we should delete the whole row
ind_back, ind_sig = idx_2labels(Y, [-1,1])
Y[ind_back] = 0

input_data, mean_X, std_X = standardize(input_data)    

-999 are replaced by the mean value of the feature


In [31]:
# Subdived the X features space in single features
all_features = np.genfromtxt(my_path + r'/data/train.csv', delimiter=",", dtype=str, max_rows = 1)[2:]
# converting array in list in order to simplify the adding of features
all_features = list(all_features)

features = []
for i in range(len(all_features)):
    features.append((i,all_features[i]))


## R^2  as stopping criteria : 1) R2 with error , 2) R2 of Tjur or 3) R2 of McFadden

The only way to use the stepwise is using R2 of Tjur or McFadden because of the binary values of the indipendent variable, but the error was also used 

In [32]:
def results_r2_stepwise(list_r2_adj,indices_features):
    print("R2 asjusted values:")
    
    for i in range(len(list_r2_adj)):
        print(list_r2_adj[i])
    print("-------------------------------------------------------")
    print("Number of features chosen:", len(indices_features))
    print("\n")
    print("Indices of features chosen: ", indices_features)
    

## A) Least square model

### No cross validation (NC)

#### 1) NC: Use of the error before binarization (R2 with loss)

In [11]:
# Start of STEP-WISE algorithm 
all_candidates = input_data

n = all_candidates.shape[0] #needed for the R^2 adjusted
num = all_candidates.shape[1]
H = np.ones((n,1)) #offset

#Initialization only with offsets (lack of info)
X = H
k = 0 #needed for the R^2 adjusted

w0, loss = least_squares(Y,X)  # The loss cannot be used as a measure for the feature selection because it's a 
y = predict_labels(w0, X)

sse = loss*2*n
sst = np.sum((Y - Y.mean())**2)  #lack of information
R2 = np.abs((sst-sse)/sst)
R2adj_0 = R2 - (k/(n-k-1)*(1-R2))

#fix the R2adj_max

R2adj_max = R2adj_0
ind_max = 0  # this index will show us which is the best feature chosen
del(X)
idx_features = []
best_R2adj = []

for j in range(num):
    R2_adj = []
    for i in range(all_candidates.shape[1]):
        
        X = np.concatenate((H,all_candidates[:,i].reshape(n,1)), axis=1)
        ws , loss = least_squares(Y,X)
        k = len(ws) -1 # k is the number of regressor I use -> -1 because I don't consider the offset
        
        #y = predict_labels(ws,X)   #***** NO USE OF PREDICTION
        SSE = loss*2*n
        SST = np.sum((Y- Y.mean())**2)
        R2 = np.abs((SST-SSE)/SST)
        R2_adj.append(R2 - (k/(n-k-1)*(1-R2)))
        
    R2adj_chosen = np.max(R2_adj)
    best_R2adj.append(R2adj_chosen)
    idx_chosen = np.argmax(R2_adj)
    
    if R2adj_chosen > R2adj_max:
        R2adj_max = R2adj_chosen
        ind_max = idx_chosen
        
        H = np.concatenate((H, all_candidates[:,ind_max].reshape(n,1)), axis = 1)
        
        all_candidates = np.delete(all_candidates,ind_max,1)
        print('-------------------------------------------------')
        print('Feature chosen: ', features[ind_max][1], '(index :', features[ind_max][0], ')')
        idx_features.append(features[ind_max][0])
        del(features[ind_max])
        
        del(X)
        
    else:
        break
        

-------------------------------------------------
Feature chosen:  DER_mass_transverse_met_lep (index : 1 )
-------------------------------------------------
Feature chosen:  PRI_tau_pt (index : 13 )
-------------------------------------------------
Feature chosen:  DER_deltaeta_jet_jet (index : 4 )
-------------------------------------------------
Feature chosen:  DER_met_phi_centrality (index : 11 )
-------------------------------------------------
Feature chosen:  DER_deltar_tau_lep (index : 7 )
-------------------------------------------------
Feature chosen:  DER_mass_vis (index : 2 )
-------------------------------------------------
Feature chosen:  PRI_lep_pt (index : 16 )
-------------------------------------------------
Feature chosen:  DER_pt_ratio_lep_tau (index : 10 )
-------------------------------------------------
Feature chosen:  PRI_met (index : 19 )
-------------------------------------------------
Feature chosen:  DER_lep_eta_centrality (index : 12 )
----------------

In [12]:
results_r2_stepwise(best_R2adj[:len(best_R2adj)-1], idx_features)

R2 asjusted values:
0.123498102139
0.158127051218
0.183285452779
0.19737862558
0.204845050856
0.212458380282
0.221835080472
0.22963272572
0.234367607585
0.237448089064
0.239839917382
0.241052424021
0.242204099894
0.243490389003
0.243684846279
0.244154585661
0.244186288257
0.244191908959
0.244195224799
0.24419603504
0.244196344677
-------------------------------------------------------
Number of features chosen: 21


Indices of features chosen:  [1, 13, 4, 11, 7, 2, 16, 10, 19, 12, 23, 8, 5, 26, 22, 21, 0, 18, 6, 3, 28]


#### 2) NC: Use of the probability of the 2 events (R2 Tjur)

In [ ]:
# Realloc FEATURES
features = []
for i in range(len(all_features)):
    features.append((i,all_features[i]))


In [ ]:
# Start of STEP-WISE algorithm 
all_candidates = input_data

n = all_candidates.shape[0] #needed for the R^2 adjusted
num = all_candidates.shape[1]
H = np.ones((n,1)) #offset

#Initialization only with offsets (lack of info)
X = H
k = 0 #needed for the R^2 adjusted

w0, loss = least_squares(Y,X) 
y = predict_labels(w0, X)
ind_back, ind_sig = idx_2labels(y, [0,1])

y_ = X.dot(w0)
R2 = 0
R2adj_0 = R2 - (k/(n-k-1)*(1-R2))

#fix the R2adj_max
R2adj_max = R2adj_0
ind_max = 0  # this index will show us which is the best feature chosen
del(X)
idx_features = []
best_R2adj = []

for j in range(num):
    R2_adj = []
    for i in range(all_candidates.shape[1]):
        
        X = np.concatenate((H,all_candidates[:,i].reshape(n,1)), axis=1)
        ws , loss = least_squares(Y,X)
        k = len(ws) -1 # k is the number of regressor I use -> -1 because I don't consider the offset
        
        y = predict_labels(ws,X)          
        ind_back, ind_sig = idx_2labels(y, [0,1])
        
        if len(ind_sig) == 0 or len(ind_back) ==0:
            print('No signal detected')
            R2_adj.append(0)
            
        else: 
            
            y_ = X.dot(ws)

            R2 = np.abs((np.mean(y_[ind_sig]) - np.mean(y_[ind_back])))
            R2_adj.append(R2 - (k/(n-k-1)*(1-R2)))
            
        
    R2adj_chosen = np.max(R2_adj)
    best_R2adj.append(R2adj_chosen)
    idx_chosen = np.argmax(R2_adj)

    if R2adj_chosen > R2adj_max:
        R2adj_max = R2adj_chosen
        ind_max = idx_chosen

        #idx_features.append(np.where(all_candidates[:,ind_max] == input_data))
        H = np.concatenate((H, all_candidates[:,ind_max].reshape(n,1)), axis = 1)

        all_candidates = np.delete(all_candidates,ind_max,1)
        print('-------------------------------------------------')
        print('Feature chosen: ', features[ind_max][1], '(index :', features[ind_max][0], ')')
        idx_features.append(features[ind_max][0])
        del(features[ind_max])

        del(X)

    else:
        break

In [ ]:
results_r2_stepwise(best_R2adj[:len(best_R2adj)-1], idx_features)

#### 3) NC: Use of the likelihood (McFadden)

In [33]:
# Realloc FEATURES
features = []
for i in range(len(all_features)):
    features.append((i,all_features[i]))
   

In [34]:
# Start of STEP-WISE algorithm 
all_candidates = input_data

n = all_candidates.shape[0] #needed for the R^2 adjusted
num = all_candidates.shape[1]
H = np.ones((n,1)) #offset

#Initialization only with offsets (lack of info)
X = H
k = 0 #needed for the R^2 adjusted

w0, loss = least_squares(Y,X)  # The loss cannot be used as a measure for the feature selection because it's a 
y = predict_labels(w0, X)
loglike0 = compute_loglikelihood_reg(y,X,w0)#np.sum(np.log(1+np.exp(X.dot(w0))) - y*(X.dot(w0)))

R2 = 0        # For the definition of McFadden 1-1 = 0
R2adj_0 = 0

#fix the R2adj_max

R2adj_max = R2adj_0
ind_max = 0  # this index will show us which is the best feature chosen
del(X)
idx_features = []
best_R2adj = []

for j in range(num):
    R2_adj = []
    for i in range(all_candidates.shape[1]):
        
        X = np.concatenate((H,all_candidates[:,i].reshape(n,1)), axis=1)
        ws , loss = least_squares(Y,X)
        k = len(ws) -1 # k is the number of regressor I use -> -1 because I don't consider the offset
        
        y = predict_labels(ws,X)   
        
        loglike = compute_loglikelihood_reg(y,X,ws) #np.sum(np.log(1+np.exp(X.dot(ws))) - y*(X.dot(ws)))
        
        R2 = 1-(loglike/loglike0)
        R2_adj.append(R2 - (k/(n-k-1)*(1-R2)))
        
    R2adj_chosen = np.max(R2_adj)
    best_R2adj.append(R2adj_chosen)
    idx_chosen = np.argmax(R2_adj)
    
    if R2adj_chosen > R2adj_max:
        R2adj_max = R2adj_chosen
        ind_max = idx_chosen
        
        H = np.concatenate((H, all_candidates[:,ind_max].reshape(n,1)), axis = 1)
        
        all_candidates = np.delete(all_candidates,ind_max,1)
        print('-------------------------------------------------')
        print('Feature chosen: ', features[ind_max][1], '(index :', features[ind_max][0], ')')
        idx_features.append(features[ind_max][0])
        del(features[ind_max])
        
        del(X)
        
    else:
        break
        

-------------------------------------------------
Feature chosen:  DER_mass_transverse_met_lep (index : 1 )
-------------------------------------------------
Feature chosen:  DER_met_phi_centrality (index : 11 )
-------------------------------------------------
Feature chosen:  DER_sum_pt (index : 9 )
-------------------------------------------------
Feature chosen:  DER_mass_vis (index : 2 )
-------------------------------------------------
Feature chosen:  PRI_lep_phi (index : 18 )
-------------------------------------------------
Feature chosen:  PRI_jet_subleading_phi (index : 28 )
-------------------------------------------------
Feature chosen:  PRI_jet_leading_eta (index : 24 )


In [35]:
results_r2_stepwise(best_R2adj[:len(best_R2adj)-1], idx_features)

R2 asjusted values:
0.127199992503
0.161949175423
0.162775033532
0.163218892521
0.163296584222
0.163326074239
0.163331726027
-------------------------------------------------------
Number of features chosen: 7


Indices of features chosen:  [1, 11, 9, 2, 18, 28, 24]


In [ ]:
loglike/loglike0

## Using cross validation (C) (TO BE DELETED)

I am not sure that cross validation can be used we don't have to estimate any hyperparameters

### 3) (C): R2 with likelihood (McFadden)

In [ ]:
from split_data import split_data

In [36]:
# Realloc FEATURES
features = []
for i in range(len(all_features)):
    features.append((i,all_features[i]))

In [37]:
# Start of STEP-WISE algorithm 
all_candidates = input_data

n = all_candidates.shape[0] #needed for the R^2 adjusted
num = all_candidates.shape[1]
H = np.ones((n,1)) #offset

#Initialization only with offsets (lack of info)
X = H
k = 0 #needed for the R^2 adjusted

w0, loss = least_squares(Y,X)  # The loss cannot be used as a measure for the feature selection because it's a 
y = predict_labels(w0, X)
loglike0 = np.sum(np.log(1+np.exp(X.dot(w0))) - y*(X.dot(w0)))

R2 = 0   # For the definition of McFadden 1-1 = 0
R2adj_0 = 0

#fix the R2adj_max

R2adj_max = R2adj_0
ind_max = 0  # this index will show us which is the best feature chosen
del(X)
idx_features = []
best_R2adj = []

# parameters for cross validation
arg_ls = dict()
arg_ls['method'] = 'ls'
arg_ls['loss'] = 'rmse'
arg_ls['k_fold'] = 10


for j in range(num):
    R2_adj = []
    for i in range(all_candidates.shape[1]):
        
        X = np.concatenate((H,all_candidates[:,i].reshape(n,1)), axis=1)
        
        # CROSS-VALIDATION
        
        w_tr_tot, loss_tr_tot, loss_te_tot = cross_validation(Y, X, arg_ls)
        
        ws = w_tr_tot[np.argmin(loss_te_tot)]
        
        k = len(ws) -1 # k is the number of regressor I use -> -1 because I don't consider the offset
        
        y = predict_labels(ws,X)   
        
        loglike = np.sum(np.log(1+np.exp(X.dot(ws))) - y*(X.dot(ws)))
        
        R2 = 1-(loglike/loglike0)
        R2_adj.append(R2 - (k/(n-k-1)*(1-R2)))
        
    R2adj_chosen = np.max(R2_adj)
    best_R2adj.append(R2adj_chosen)
    idx_chosen = np.argmax(R2_adj)
    
    if R2adj_chosen > R2adj_max:
        R2adj_max = R2adj_chosen
        ind_max = idx_chosen
        
        H = np.concatenate((H, all_candidates[:,ind_max].reshape(n,1)), axis = 1)
        
        all_candidates = np.delete(all_candidates,ind_max,1)
        print('-------------------------------------------------')
        print('Feature chosen: ', features[ind_max][1], '(index :', features[ind_max][0], ')')
        idx_features.append(features[ind_max][0])
        del(features[ind_max])
        
        del(X)
        
    else:
        break

-------------------------------------------------
Feature chosen:  DER_mass_transverse_met_lep (index : 1 )
-------------------------------------------------
Feature chosen:  DER_met_phi_centrality (index : 11 )
-------------------------------------------------
Feature chosen:  DER_sum_pt (index : 9 )
-------------------------------------------------
Feature chosen:  PRI_jet_all_pt (index : 29 )
-------------------------------------------------
Feature chosen:  PRI_met (index : 19 )
-------------------------------------------------
Feature chosen:  DER_pt_h (index : 3 )
-------------------------------------------------
Feature chosen:  DER_lep_eta_centrality (index : 12 )
-------------------------------------------------
Feature chosen:  DER_mass_MMC (index : 0 )
-------------------------------------------------
Feature chosen:  DER_deltar_tau_lep (index : 7 )
-------------------------------------------------
Feature chosen:  DER_pt_ratio_lep_tau (index : 10 )
-------------------------

In [38]:
results_r2_stepwise(best_R2adj[:len(best_R2adj)-1], idx_features)

R2 asjusted values:
0.127029003781
0.162219139573
0.163016309637
0.16354554708
0.167617483869
0.172882697354
0.174062939737
0.174659663743
0.175157077078
0.176255853643
0.177589693328
0.17903001667
0.180289605671
0.180653688622
0.180673303474
0.18068436144
-------------------------------------------------------
Number of features chosen: 16


Indices of features chosen:  [1, 11, 9, 29, 19, 3, 12, 0, 7, 10, 2, 16, 23, 21, 15, 13]


# B) Ridge regression

I used only lambda as hyperparameter because I am not building a polynomial model. But we can add different features transformation (square or log or power), so maybe we can test the polynomial after the best features are selected 

## No cross validation (NC)

In [16]:
# set lambda
lambda_ = 0.05

### 1) (NC) Using the loss from ridge regression

In [17]:
# Realloc FEATURES
features = []
for i in range(len(all_features)):
    features.append((i,all_features[i]))


In [18]:
# Start of STEP-WISE algorithm 
all_candidates = input_data

n = all_candidates.shape[0] #needed for the R^2 adjusted
num = all_candidates.shape[1]
H = np.ones((n,1)) #offset

#Initialization only with offsets (lack of info)
X = H
k = 0 #needed for the R^2 adjusted

w0, loss = ridge_regression(Y,X,0)  # start with lambda = 0  
y = predict_labels(w0, X)

sse = loss*2*n
sst = np.sum((Y - Y.mean())**2)  #lack of information
R2 = np.abs((sst-sse)/sst)
R2adj_0 = R2 - (k/(n-k-1)*(1-R2))

#fix the R2adj_max

R2adj_max = R2adj_0
ind_max = 0  # this index will show us which is the best feature chosen
del(X)
idx_features = []
best_R2adj = []

for j in range(num):
    R2_adj = []
    for i in range(all_candidates.shape[1]):
        
        X = np.concatenate((H,all_candidates[:,i].reshape(n,1)), axis=1)
        
        ws, loss = ridge_regression(Y,X,lambda_)
        k = len(ws) -1 # k is the number of regressor I use -> -1 because I don't consider the offset
        
        SSE = loss*2*n
        SST = np.sum((Y- Y.mean())**2)   # it has no sense
        R2 = np.abs((SST-SSE)/SST)
        R2_adj.append(R2 - (k/(n-k-1)*(1-R2)))
        
    R2adj_chosen = np.max(R2_adj)
    best_R2adj.append(R2adj_chosen)
    idx_chosen = np.argmax(R2_adj)
    
    if R2adj_chosen > R2adj_max:
        R2adj_max = R2adj_chosen
        ind_max = idx_chosen
        
        H = np.concatenate((H, all_candidates[:,ind_max].reshape(n,1)), axis = 1)
        
        all_candidates = np.delete(all_candidates,ind_max,1)
        print('-------------------------------------------------')
        print('Feature chosen: ', features[ind_max][1], '(index :', features[ind_max][0], ')')
        idx_features.append(features[ind_max][0])
        del(features[ind_max])
        
        del(X)
        
    else:
        break
        

-------------------------------------------------
Feature chosen:  DER_mass_transverse_met_lep (index : 1 )
-------------------------------------------------
Feature chosen:  PRI_tau_pt (index : 13 )
-------------------------------------------------
Feature chosen:  DER_deltaeta_jet_jet (index : 4 )
-------------------------------------------------
Feature chosen:  DER_met_phi_centrality (index : 11 )
-------------------------------------------------
Feature chosen:  DER_deltar_tau_lep (index : 7 )
-------------------------------------------------
Feature chosen:  DER_lep_eta_centrality (index : 12 )
-------------------------------------------------
Feature chosen:  PRI_met (index : 19 )
-------------------------------------------------
Feature chosen:  DER_mass_vis (index : 2 )
-------------------------------------------------
Feature chosen:  DER_mass_jet_jet (index : 5 )
-------------------------------------------------
Feature chosen:  PRI_jet_subleading_pt (index : 26 )
----------

In [19]:
results_r2_stepwise(best_R2adj[:len(best_R2adj)-1], idx_features)

R2 asjusted values:
0.0590175831678
0.0919564321754
0.115305280064
0.131172460715
0.137081038671
0.14163931635
0.143708108081
0.145592367705
0.146688011854
0.148345611287
0.148987515541
0.149696003179
0.14984311946
0.149851916892
0.149853409866
-------------------------------------------------------
Number of features chosen: 15


Indices of features chosen:  [1, 13, 4, 11, 7, 12, 19, 2, 5, 26, 8, 3, 22, 18, 15]


### 2) (NC) Use of the probability of the 2 events (R2 Tjur)

In [ ]:
# Realloc FEATURES
features = []
for i in range(len(all_features)):
    features.append((i,all_features[i]))

In [ ]:
# Start of STEP-WISE algorithm 
all_candidates = input_data

n = all_candidates.shape[0] #needed for the R^2 adjusted
num = all_candidates.shape[1]
H = np.ones((n,1)) #offset

#Initialization only with offsets (lack of info)
X = H
k = 0 #needed for the R^2 adjusted

w0, loss = ridge_regression(Y,X,0)  
y = predict_labels(w0, X)
ind_back, ind_sig = idx_2labels(y, [0,1])

y_ = X.dot(w0)
R2 = 0
R2adj_0 = R2 - (k/(n-k-1)*(1-R2))

#fix the R2adj_max
R2adj_max = R2adj_0
ind_max = 0  # this index will show us which is the best feature chosen
del(X)
idx_features = []
best_R2adj = []

for j in range(num):
    R2_adj = []
    for i in range(all_candidates.shape[1]):
        
        X = np.concatenate((H,all_candidates[:,i].reshape(n,1)), axis=1)
        ws, loss = ridge_regression(Y,X,lambda_)
        
        k = len(ws) -1 # k is the number of regressor I use -> -1 because I don't consider the offset
        
        y = predict_labels(ws,X)          
        ind_back, ind_sig = idx_2labels(y, [-1,1])
        
        if len(ind_sig) == 0 or len(ind_back) ==0:
            print('No signal detected')
            R2_adj.append(0)
            
        else: 
            
            y_ = X.dot(ws)

            R2 = np.abs((np.mean(y_[ind_sig]) - np.mean(y_[ind_back])))
            R2_adj.append(R2 - (k/(n-k-1)*(1-R2)))
            
        
    R2adj_chosen = np.max(R2_adj)
    best_R2adj.append(R2adj_chosen)
    idx_chosen = np.argmax(R2_adj)

    if R2adj_chosen > R2adj_max:
        R2adj_max = R2adj_chosen
        ind_max = idx_chosen

        #idx_features.append(np.where(all_candidates[:,ind_max] == input_data))
        H = np.concatenate((H, all_candidates[:,ind_max].reshape(n,1)), axis = 1)

        all_candidates = np.delete(all_candidates,ind_max,1)
        print('-------------------------------------------------')
        print('Feature chosen: ', features[ind_max][1], '(index :', features[ind_max][0], ')')
        idx_features.append(features[ind_max][0])
        del(features[ind_max])

        del(X)

    else:
        break

In [ ]:
results_r2_stepwise(best_R2adj[:len(best_R2adj)-1], idx_features)

### 3) (NC) Use of the likelihood (McFadden)

In [20]:
# Realloc FEATURES
features = []
for i in range(len(all_features)):
    features.append((i,all_features[i]))

In [21]:
# Start of STEP-WISE algorithm 
all_candidates = input_data

n = all_candidates.shape[0] #needed for the R^2 adjusted
num = all_candidates.shape[1]
H = np.ones((n,1)) #offset

#Initialization only with offsets (lack of info)
X = H
k = 0 #needed for the R^2 adjusted

w0, loss = ridge_regression(Y,X,0)  # lambda set to 0 
y = predict_labels(w0, X)
loglike0 = compute_loglikelihood_reg(y,X,w0) #np.sum(np.log(1+np.exp(X.dot(w0))) - y*(X.dot(w0)))

R2 = 0        # For the definition of McFadden 1-1 = 0
R2adj_0 = 0

#fix the R2adj_max

R2adj_max = R2adj_0
ind_max = 0  # this index will show us which is the best feature chosen
del(X)
idx_features = []
best_R2adj = []

for j in range(num):
    R2_adj = []
    for i in range(all_candidates.shape[1]):
        
        X = np.concatenate((H,all_candidates[:,i].reshape(n,1)), axis=1)
        
        ws, loss = ridge_regression(Y,X,lambda_)
        
        k = len(ws) -1 # k is the number of regressor I use -> -1 because I don't consider the offset
        
        y = predict_labels(ws,X)   
        
        loglike = compute_loglikelihood_reg(y,X,ws) #np.sum(np.log(1+np.exp(X.dot(ws))) - y*(X.dot(ws)))
        
        R2 = 1-(loglike/loglike0)
        R2_adj.append(R2 - (k/(n-k-1)*(1-R2)))
        
    R2adj_chosen = np.max(R2_adj)
    best_R2adj.append(R2adj_chosen)
    idx_chosen = np.argmax(R2_adj)
    
    if R2adj_chosen > R2adj_max:
        R2adj_max = R2adj_chosen
        ind_max = idx_chosen
        
        H = np.concatenate((H, all_candidates[:,ind_max].reshape(n,1)), axis = 1)
        
        all_candidates = np.delete(all_candidates,ind_max,1)
        print('-------------------------------------------------')
        print('Feature chosen: ', features[ind_max][1], '(index :', features[ind_max][0], ')')
        idx_features.append(features[ind_max][0])
        del(features[ind_max])
        
        del(X)
        
    else:
        break
        

-------------------------------------------------
Feature chosen:  DER_mass_transverse_met_lep (index : 1 )
-------------------------------------------------
Feature chosen:  DER_met_phi_centrality (index : 11 )
-------------------------------------------------
Feature chosen:  DER_sum_pt (index : 9 )
-------------------------------------------------
Feature chosen:  DER_mass_vis (index : 2 )
-------------------------------------------------
Feature chosen:  PRI_lep_phi (index : 18 )
-------------------------------------------------
Feature chosen:  PRI_jet_subleading_phi (index : 28 )
-------------------------------------------------
Feature chosen:  PRI_jet_leading_eta (index : 24 )


In [22]:
results_r2_stepwise(best_R2adj[:len(best_R2adj)-1], idx_features)

R2 asjusted values:
0.127200013384
0.161949192879
0.162775051476
0.163218910102
0.163296601788
0.1633260918
0.163331743587
-------------------------------------------------------
Number of features chosen: 7


Indices of features chosen:  [1, 11, 9, 2, 18, 28, 24]


## With cross validation (C) (it's needed in order to choose the best hyperparameter lambda)

### 1) (C) Using the loss of ridge regression as the error of R2 

In [25]:
# Realloc FEATURES
features = []
for i in range(len(all_features)):
    features.append((i,all_features[i]))


In [26]:
# Start of STEP-WISE algorithm 
all_candidates = input_data

n = all_candidates.shape[0] #needed for the R^2 adjusted
num = all_candidates.shape[1]
H = np.ones((n,1)) #offset

#Initialization only with offsets (lack of info)
X = H
k = 0 #needed for the R^2 adjusted

w0, loss = ridge_regression(Y,X,0)  # start with lambda = 0  
y = predict_labels(w0, X)

sse = loss*2*n
sst = np.sum((Y - Y.mean())**2)  #lack of information
R2 = np.abs((sst-sse)/sst)
R2adj_0 = R2 - (k/(n-k-1)*(1-R2))

#fix the R2adj_max
R2adj_max = R2adj_0
print("R2adj_0= ", R2adj_0)
ind_max = 0  # this index will show us which is the best feature chosen
del(X)
idx_features = []
best_R2adj = []

# ridge regression parameters
arg_rr = dict()
arg_rr['method'] = 'rr'
arg_rr['loss'] = 'rmse'
arg_rr['degree'] = 1
arg_rr['k_fold'] = 10

# optimization parameters
lambda_min = -10 
lambda_max = 1
lambda_steps = 10

for j in range(num):
    R2_adj = []
    for i in range(all_candidates.shape[1]):
        
        X = np.concatenate((H,all_candidates[:,i].reshape(n,1)), axis=1)
        
        ##########################################CROSS VALIDATION##########################################
        
        ws, loss_tr, loss, lambda_opt = optimize_lambda(Y, X, lambda_min, lambda_max, lambda_steps, arg_rr)
        
        ####################################################################################################
        
        #ws = w_tr_tot[np.argmin(loss_te_tot)]
        #loss = np.min(loss_te_tot)
        
        k = len(ws) -1 # k is the number of regressor I use -> -1 because I don't consider the offset
        
        SSE = loss*2*n
        SST = np.sum((Y- Y.mean())**2)
        R2 = np.abs((SST-SSE)/SST)
        R2_adj.append(R2 - (k/(n-k-1)*(1-R2)))
        
        print("SSE:", SSE, " SST:", SST, " R2", R2, "R2_adj: ", R2_adj)
        
    R2adj_chosen = np.max(R2_adj)
    best_R2adj.append(R2adj_chosen)
    idx_chosen = np.argmax(R2_adj)
    
    if R2adj_chosen > R2adj_max:
        R2adj_max = R2adj_chosen
        ind_max = idx_chosen
        
        H = np.concatenate((H, all_candidates[:,ind_max].reshape(n,1)), axis = 1)
        
        all_candidates = np.delete(all_candidates,ind_max,1)
        print('-------------------------------------------------')
        print('Feature chosen: ', features[ind_max][1], '(index :', features[ind_max][0], ')')
        idx_features.append(features[ind_max][0])
        del(features[ind_max])
        
        del(X)
        
    else:
        break
        

R2adj_0=  5.22003232182e-14
SSE: 237291.57955  SST: 56311.660444  R2 3.21389775544 R2_adj:  [3.2139066111045418]
SSE: 222166.97139  SST: 56311.660444  R2 2.94531025437 R2_adj:  [3.2139066111045418, 2.9453180356717179]
SSE: 237278.684431  SST: 56311.660444  R2 3.21366875991 R2_adj:  [3.2139066111045418, 2.9453180356717179, 3.2136776146527186]
SSE: 232862.58042  SST: 56311.660444  R2 3.13524620982 R2_adj:  [3.2139066111045418, 2.9453180356717179, 3.2136776146527186, 3.1352547508779942]
SSE: 233286.64172  SST: 56311.660444  R2 3.14277682244 R2_adj:  [3.2139066111045418, 2.9453180356717179, 3.2136776146527186, 3.1352547508779942, 3.1427853936117813]
SSE: 233563.179252  SST: 56311.660444  R2 3.14768766203 R2_adj:  [3.2139066111045418, 2.9453180356717179, 3.2136776146527186, 3.1352547508779942, 3.1427853936117813, 3.1476962528492343]
SSE: 234083.688294  SST: 56311.660444  R2 3.15693102368 R2_adj:  [3.2139066111045418, 2.9453180356717179, 3.2136776146527186, 3.1352547508779942, 3.142785393611

SSE: 237279.890767  SST: 56311.660444  R2 3.21369018239 R2_adj:  [3.2139066111045418, 2.9453180356717179, 3.2136776146527186, 3.1352547508779942, 3.1427853936117813, 3.1476962528492343, 3.1569396514684209, 3.2137792399633129, 3.2136177339159024, 3.1643291803372549, 3.1328631810319014, 3.0555014888334062, 3.1511369636591851, 3.0958441712067239, 3.2141062931440474, 3.2140471197146332, 3.2119414729836446, 3.2140923824249508, 3.2140741520292671, 3.2130315384843251, 3.2139788149441491, 3.1752203551391895, 3.1763413877188031, 3.2084735350236708, 3.2140989582273543, 3.2140910249122383, 3.2136990372255343]
SSE: 237302.27354  SST: 56311.660444  R2 3.21408766264 R2_adj:  [3.2139066111045418, 2.9453180356717179, 3.2136776146527186, 3.1352547508779942, 3.1427853936117813, 3.1476962528492343, 3.1569396514684209, 3.2137792399633129, 3.2136177339159024, 3.1643291803372549, 3.1328631810319014, 3.0555014888334062, 3.1511369636591851, 3.0958441712067239, 3.2141062931440474, 3.2140471197146332, 3.2119414

SSE: 235177.600888  SST: 56311.660444  R2 3.17635706413 R2_adj:  [3.2139424352927648, 2.9453540089653028, 3.2137135124393912, 3.1352895995859744, 3.1428236881254494, 3.1477356952247733, 3.1569767516542502, 3.2138151073415058, 3.2136536771010751, 3.1643665078920646, 3.1328971201825397, 3.0555294830601172, 3.1511743036216071, 3.0958793010960903, 3.214083037164909, 3.2119769285380717, 3.2141236169308138, 3.2141099990571322, 3.213067085271093, 3.2140148266580777, 3.1752570815513192, 3.1763744751937701]
SSE: 236987.20262  SST: 56311.660444  R2 3.20849253513 R2_adj:  [3.2139424352927648, 2.9453540089653028, 3.2137135124393912, 3.1352895995859744, 3.1428236881254494, 3.1477356952247733, 3.1569767516542502, 3.2138151073415058, 3.2136536771010751, 3.1643665078920646, 3.1328971201825397, 3.0555294830601172, 3.1511743036216071, 3.0958793010960903, 3.214083037164909, 3.2119769285380717, 3.2141236169308138, 3.2141099990571322, 3.213067085271093, 3.2140148266580777, 3.1752570815513192, 3.17637447519

SSE: 237304.36599  SST: 56311.660444  R2 3.21412482102 R2_adj:  [3.2139700458569354, 2.9453788333770152, 3.2137407445396029, 3.1353160421843422, 3.1428495940939847, 3.1477596318845769, 3.1570036060331566, 3.2138427583655496, 3.213681195188784, 3.1643925524092329, 3.132922868644699, 3.0555541025982076, 3.1511992467632486, 3.0959095054070898, 3.214110415647514, 3.2120040758204782, 3.2141513909444517]
SSE: 237303.577353  SST: 56311.660444  R2 3.21411081616 R2_adj:  [3.2139700458569354, 2.9453788333770152, 3.2137407445396029, 3.1353160421843422, 3.1428495940939847, 3.1477596318845769, 3.1570036060331566, 3.2138427583655496, 3.213681195188784, 3.1643925524092329, 3.132922868644699, 3.0555541025982076, 3.1511992467632486, 3.0959095054070898, 3.214110415647514, 3.2120040758204782, 3.2141513909444517, 3.2141373859142748]
SSE: 237244.862739  SST: 56311.660444  R2 3.21306814376 R2_adj:  [3.2139700458569354, 2.9453788333770152, 3.2137407445396029, 3.1353160421843422, 3.1428495940939847, 3.1477596

SSE: 228374.161168  SST: 56311.660444  R2 3.05553946318 R2_adj:  [3.2139948650084702, 2.9453973737867791, 3.2137656789446689, 3.13533771212684, 3.1428695736161325, 3.1477841990477526, 3.1570367789266376, 3.2138675305082494, 3.2137059479557539, 3.1644158243220768, 3.132949845500987, 3.0555723524697087]
SSE: 233760.58421  SST: 56311.660444  R2 3.15119324074 R2_adj:  [3.2139948650084702, 2.9453973737867791, 3.2137656789446689, 3.13533771212684, 3.1428695736161325, 3.1477841990477526, 3.1570367789266376, 3.2138675305082494, 3.2137059479557539, 3.1644158243220768, 3.132949845500987, 3.0555723524697087, 3.1512276605223968]
SSE: 230647.034471  SST: 56311.660444  R2 3.095901855 R2_adj:  [3.2139948650084702, 2.9453973737867791, 3.2137656789446689, 3.13533771212684, 3.1428695736161325, 3.1477841990477526, 3.1570367789266376, 3.2138675305082494, 3.2137059479557539, 3.1644158243220768, 3.132949845500987, 3.0555723524697087, 3.1512276605223968, 3.095935390098302]
SSE: 237302.963643  SST: 56311.6604

SSE: 237288.541022  SST: 56311.660444  R2 3.21384379631 R2_adj:  [3.2140155529478904, 2.9454169233524161, 3.2137863923653791, 3.1353568139114292, 3.142889312163637, 3.1478036211241518, 3.157056366051068, 3.2138880742485627]
SSE: 237279.467003  SST: 56311.660444  R2 3.21368265706 R2_adj:  [3.2140155529478904, 2.9454169233524161, 3.2137863923653791, 3.1353568139114292, 3.142889312163637, 3.1478036211241518, 3.157056366051068, 3.2138880742485627, 3.213726931774687]
SSE: 234503.782048  SST: 56311.660444  R2 3.16439117936 R2_adj:  [3.2140155529478904, 2.9454169233524161, 3.2137863923653791, 3.1353568139114292, 3.142889312163637, 3.1478036211241518, 3.157056366051068, 3.2138880742485627, 3.213726931774687, 3.1644344682178795]
SSE: 232732.345354  SST: 56311.660444  R2 3.13293345497 R2_adj:  [3.2140155529478904, 2.9454169233524161, 3.2137863923653791, 3.1353568139114292, 3.142889312163637, 3.1478036211241518, 3.157056366051068, 3.2138880742485627, 3.213726931774687, 3.1644344682178795, 3.13297

SSE: 233290.998872  SST: 56311.660444  R2 3.1428541981 R2_adj:  [3.2140346935184825, 2.9454422469518819, 3.2138055003670538, 3.1353824254071343, 3.1429056280368224]
SSE: 233567.977769  SST: 56311.660444  R2 3.14777287559 R2_adj:  [3.2140346935184825, 2.9454422469518819, 3.2138055003670538, 3.1353824254071343, 3.1429056280368224, 3.1478244235847206]
SSE: 234088.831  SST: 56311.660444  R2 3.15702234945 R2_adj:  [3.2140346935184825, 2.9454422469518819, 3.2138055003670538, 3.1353824254071343, 3.1429056280368224, 3.1478244235847206, 3.1570741194381151]
SSE: 237289.129297  SST: 56311.660444  R2 3.21385424309 R2_adj:  [3.2140346935184825, 2.9454422469518819, 3.2138055003670538, 3.1353824254071343, 3.1429056280368224, 3.1478244235847206, 3.1570741194381151, 3.2139073770762789]
SSE: 237279.973393  SST: 56311.660444  R2 3.21369164969 R2_adj:  [3.2140346935184825, 2.9454422469518819, 3.2138055003670538, 3.1353824254071343, 3.1429056280368224, 3.1478244235847206, 3.1570741194381151, 3.213907377076

SSE: 233290.287702  SST: 56311.660444  R2 3.14284156892 R2_adj:  [3.2140428657291431, 2.9454648009079354, 3.2138135488638162, 3.1354019084094387, 3.1429015704020444]
SSE: 233567.829938  SST: 56311.660444  R2 3.14777025036 R2_adj:  [3.2140428657291431, 2.9454648009079354, 3.2138135488638162, 3.1354019084094387, 3.1429015704020444, 3.1478303898519195]
SSE: 234088.591374  SST: 56311.660444  R2 3.15701809409 R2_adj:  [3.2140428657291431, 2.9454648009079354, 3.2138135488638162, 3.1354019084094387, 3.1429015704020444, 3.1478303898519195, 3.1570784925328863]
SSE: 237289.033377  SST: 56311.660444  R2 3.21385253971 R2_adj:  [3.2140428657291431, 2.9454648009079354, 3.2138135488638162, 3.1354019084094387, 3.1429015704020444, 3.1478303898519195, 3.1570784925328863, 3.2139145295690339]
SSE: 237279.908004  SST: 56311.660444  R2 3.21369048849 R2_adj:  [3.2140428657291431, 2.9454648009079354, 3.2138135488638162, 3.1354019084094387, 3.1429015704020444, 3.1478303898519195, 3.1570784925328863, 3.21391452

SSE: 237288.759559  SST: 56311.660444  R2 3.21384767716 R2_adj:  [3.2140466263443717, 2.9454523855118704, 3.2138172831869456, 3.1353937494100537, 3.1429060642818603, 3.1478264562802281, 3.1570823796970817, 3.2139185228384366]
SSE: 237279.687876  SST: 56311.660444  R2 3.21368657938 R2_adj:  [3.2140466263443717, 2.9454523855118704, 3.2138172831869456, 3.1353937494100537, 3.1429060642818603, 3.1478264562802281, 3.1570823796970817, 3.2139185228384366, 3.2137574199027394]
SSE: 234504.731214  SST: 56311.660444  R2 3.16440803495 R2_adj:  [3.2140466263443717, 2.9454523855118704, 3.2138172831869456, 3.1353937494100537, 3.1429060642818603, 3.1478264562802281, 3.1570823796970817, 3.2139185228384366, 3.2137574199027394, 3.1644772984987726]
SSE: 232733.806149  SST: 56311.660444  R2 3.13295939622 R2_adj:  [3.2140466263443717, 2.9454523855118704, 3.2138172831869456, 3.1353937494100537, 3.1429060642818603, 3.1478264562802281, 3.1570823796970817, 3.2139185228384366, 3.2137574199027394, 3.16447729849877

In [ ]:
results_r2_stepwise(best_R2adj[:len(best_R2adj)-1], idx_features)

In [ ]:
R2adj_0

### 2) (C) Use of the probability of the 2 events (R2 Tjur)

In [ ]:
# Realloc FEATURES
features = []
for i in range(len(all_features)):
    features.append((i,all_features[i]))

In [ ]:
# Start of STEP-WISE algorithm 
all_candidates = input_data

n = all_candidates.shape[0] #needed for the R^2 adjusted
num = all_candidates.shape[1]
H = np.ones((n,1)) #offset

#Initialization only with offsets (lack of info)
X = H
k = 0 #needed for the R^2 adjusted

w0, loss = ridge_regression(Y,X,0)  
y = predict_labels(w0, X)
ind_back, ind_sig = idx_2labels(y, [0,1])

y_ = X.dot(w0)
R2 = 0
R2adj_0 = R2 - (k/(n-k-1)*(1-R2))

#fix the R2adj_max
R2adj_max = R2adj_0
ind_max = 0  # this index will show us which is the best feature chosen
del(X)
idx_features = []
best_R2adj = []

for j in range(num):
    R2_adj = []
    for i in range(all_candidates.shape[1]):
        
        X = np.concatenate((H,all_candidates[:,i].reshape(n,1)), axis=1)
        #CROSS VALIDATION
        w_tr_tot, loss_tr_tot, loss_te_tot = cross_validation_rr(Y,X)
        ws = w_tr_tot[np.argmin(loss_te_tot)]
        
        k = len(ws) -1 # k is the number of regressor I use -> -1 because I don't consider the offset
        
        y = predict_labels(ws,X)          
        ind_back, ind_sig = idx_2labels(y, [-1,1])
        
        if len(ind_sig) == 0 or len(ind_back) ==0:
            print('No signal detected')
            R2_adj.append(0)
            
        else: 
            
            y_ = X.dot(ws)

            R2 = np.abs((np.mean(y_[ind_sig]) - np.mean(y_[ind_back])))
            R2_adj.append(R2 - (k/(n-k-1)*(1-R2)))
            
        
    R2adj_chosen = np.max(R2_adj)
    best_R2adj.append(R2adj_chosen)
    idx_chosen = np.argmax(R2_adj)

    if R2adj_chosen > R2adj_max:
        R2adj_max = R2adj_chosen
        ind_max = idx_chosen

        #idx_features.append(np.where(all_candidates[:,ind_max] == input_data))
        H = np.concatenate((H, all_candidates[:,ind_max].reshape(n,1)), axis = 1)

        all_candidates = np.delete(all_candidates,ind_max,1)
        print('-------------------------------------------------')
        print('Feature chosen: ', features[ind_max][1], '(index :', features[ind_max][0], ')')
        idx_features.append(features[ind_max][0])
        del(features[ind_max])

        del(X)

    else:
        break

In [ ]:
results_r2_stepwise(best_R2adj[:len(best_R2adj)-1], idx_features)

### 3) (C) Use of the likelihood (McFadden)

In [ ]:
# Realloc FEATURES
features = []
for i in range(len(all_features)):
    features.append((i,all_features[i]))

In [ ]:
# Start of STEP-WISE algorithm 
all_candidates = input_data

n = all_candidates.shape[0] #needed for the R^2 adjusted
num = all_candidates.shape[1]
H = np.ones((n,1)) #offset

#Initialization only with offsets (lack of info)
X = H
k = 0 #needed for the R^2 adjusted

w0, loss = ridge_regression(Y,X,0)  # lambda set to 0 
y = predict_labels(w0, X)
loglike0 = np.sum(np.log(1+np.exp(X.dot(w0))) - y*(X.dot(w0)))

R2 = 0        # For the definition of McFadden 1-1 = 0
R2adj_0 = 0

#fix the R2adj_max

R2adj_max = R2adj_0
ind_max = 0  # this index will show us which is the best feature chosen
del(X)
idx_features = []
best_R2adj = []

for j in range(num):
    R2_adj = []
    for i in range(all_candidates.shape[1]):
        
        X = np.concatenate((H,all_candidates[:,i].reshape(n,1)), axis=1)
        #CROSS VALIDATION
        w_tr_tot, loss_tr_tot, loss_te_tot = cross_validation_rr(Y,X)
        ws = w_tr_tot[np.argmin(loss_te_tot)]
        
        k = len(ws) -1 # k is the number of regressor I use -> -1 because I don't consider the offset
        
        y = predict_labels(ws,X)   
        
        loglike = np.sum(np.log(1+np.exp(X.dot(ws))) - y*(X.dot(ws)))
        
        R2 = 1-(loglike/loglike0)
        R2_adj.append(R2 - (k/(n-k-1)*(1-R2)))
        
    R2adj_chosen = np.max(R2_adj)
    best_R2adj.append(R2adj_chosen)
    idx_chosen = np.argmax(R2_adj)
    
    if R2adj_chosen > R2adj_max:
        R2adj_max = R2adj_chosen
        ind_max = idx_chosen
        
        H = np.concatenate((H, all_candidates[:,ind_max].reshape(n,1)), axis = 1)
        
        all_candidates = np.delete(all_candidates,ind_max,1)
        print('-------------------------------------------------')
        print('Feature chosen: ', features[ind_max][1], '(index :', features[ind_max][0], ')')
        idx_features.append(features[ind_max][0])
        del(features[ind_max])
        
        del(X)
        
    else:
        break
        

In [ ]:
results_r2_stepwise(best_R2adj[:len(best_R2adj)-1], idx_features)

## Logistic regression

### No cross validation (hyperparameter fixed)

In [ ]:
# Initialization
max_iters = 100
threshold = 1e-8
gamma = 0
method = 'gd'

#### Use of the likelihood (McFadden)

I used only this method because it's the most reasonable

In [ ]:
# Realloc FEATURES
features = []
for i in range(len(all_features)):
    features.append((i,all_features[i]))

In [ ]:
# Start of STEP-WISE algorithm 
all_candidates = input_data

n = all_candidates.shape[0] #needed for the R^2 adjusted
num = all_candidates.shape[1]
H = np.ones((n,1)) #offset

#Initialization only with offsets (lack of info)
X = H
k = 0 #needed for the R^2 adjusted

initial_w = np.zeros(X.shape[1])

w0, loss = logistic_regression(Y,X, initial_w, max_iters, gamma, method)
y = predict_labels(w0[-1], X)
loglike0 = np.sum(np.log(1+np.exp(X.dot(w0))) - y*(X.dot(w0)))

R2 = 0        # For the definition of McFadden 1-1 = 0
R2adj_0 = 0

#fix the R2adj_max

R2adj_max = R2adj_0
ind_max = 0  # this index will show us which is the best feature chosen
del(X)
idx_features = []
best_R2adj = []

for j in range(num):
    R2_adj = []
    for i in range(all_candidates.shape[1]):
        
        X = np.concatenate((H,all_candidates[:,i].reshape(n,1)), axis=1)
        
        initial_w = np.zeros(X.shape[1])
        ws, loss = logistic_regression(Y,X, initial_w, max_iters, gamma, method)
        
        k = len(ws) -1 # k is the number of regressor I use -> -1 because I don't consider the offset
        
        y = predict_labels(ws[-1], X)
        loglike = compute_loglikelihood_reg(y, X, ws)
        
        R2 = 1-(loglike/loglike0)
        R2_adj.append(R2 - (k/(n-k-1)*(1-R2)))
        
    R2adj_chosen = np.max(R2_adj)
    best_R2adj.append(R2adj_chosen)
    idx_chosen = np.argmax(R2_adj)
    
    if R2adj_chosen > R2adj_max:
        R2adj_max = R2adj_chosen
        ind_max = idx_chosen
        
        H = np.concatenate((H, all_candidates[:,ind_max].reshape(n,1)), axis = 1)
        
        all_candidates = np.delete(all_candidates,ind_max,1)
        print('-------------------------------------------------')
        print('Feature chosen: ', features[ind_max][1], '(index :', features[ind_max][0], ')')
        idx_features.append(features[ind_max][0])
        del(features[ind_max])
        
        del(X)
        
    else:
        break

In [ ]:
results_r2_stepwise(best_R2adj[:len(best_R2adj)-1], idx_features)

### With cross correlation

#### Use of the likelihood (McFadden)

In [ ]:
# Realloc FEATURES
features = []
for i in range(len(all_features)):
    features.append((i,all_features[i]))

In [ ]:
# Start of STEP-WISE algorithm 
all_candidates = input_data

n = all_candidates.shape[0] #needed for the R^2 adjusted
num = all_candidates.shape[1]
H = np.ones((n,1)) #offset

#Initialization only with offsets (lack of info)
X = H
k = 0 #needed for the R^2 adjusted

w0, loss = logistic_regression(Y,X, initial_w, max_iters, gamma, method)
y = predict_labels(w0, X)
loglike0 = np.sum(np.log(1+np.exp(X.dot(w0))) - y*(X.dot(w0)))

R2 = 0        # For the definition of McFadden 1-1 = 0
R2adj_0 = 0

#fix the R2adj_max

R2adj_max = R2adj_0
ind_max = 0  # this index will show us which is the best feature chosen
del(X)
idx_features = []
best_R2adj = []

for j in range(num):
    R2_adj = []
    for i in range(all_candidates.shape[1]):
        
        X = np.concatenate((H,all_candidates[:,i].reshape(n,1)), axis=1)
        #CROSS VALIDATION
        w_tr_tot, loss_tr_tot, loss_te_tot = cross_validation_lr(Y,X)
        ws = w_tr_tot[np.argmin(loss_te_tot)]
        
        y = predict_labels(ws,X)   
        
        loglike = np.sum(np.log(1+np.exp(X.dot(ws))) - y*(X.dot(ws)))
        
        R2 = 1-(loglike/loglike0)
        R2_adj.append(R2 - (k/(n-k-1)*(1-R2)))
        
    R2adj_chosen = np.max(R2_adj)
    best_R2adj.append(R2adj_chosen)
    idx_chosen = np.argmax(R2_adj)
    
    if R2adj_chosen > R2adj_max:
        R2adj_max = R2adj_chosen
        ind_max = idx_chosen
        
        H = np.concatenate((H, all_candidates[:,ind_max].reshape(n,1)), axis = 1)
        
        all_candidates = np.delete(all_candidates,ind_max,1)
        print('-------------------------------------------------')
        print('Feature chosen: ', features[ind_max][1], '(index :', features[ind_max][0], ')')
        idx_features.append(features[ind_max][0])
        del(features[ind_max])
        
        del(X)
        
    else:
        break
        

In [ ]:
results_r2_stepwise(best_R2adj[:len(best_R2adj)-1], idx_features)